## Mongo Basic

In [ ]:
# insert movies
db.movies.insert({"id":10,"name": "titanic","Rating": "R"})

# select data
db.movies.find()

# update data
db.movies.update(
{id: 10}, {$set: {Rating: "NR"}}
)

# reove data from movies
db.movies.remove({"Rating": "NR"})

## Mongo Operation On Zip Code Dataset

In [ ]:
mongoimport --db test --collection zips --drop --file zips.json
mongo

In [ ]:
# 切換資料庫
use test

# 存取第一筆資料
db.zips.findOne()

# 使用條件篩選
db.zips.find({state:"MA"})
db.zips.find({state:"MA", , pop: { $gt : 10000}})

## Return States with Populations above 10 Million
# select state, sum(pop) from zips gorup by state having sum(pop) > 10000000
db.zips.aggregate( [
   { $group: { _id: "$state", totalPop: { $sum: "$pop" } } },
   { $match: { totalPop: { $gte: 10000000 } } }
] )

## Return Average City Population by State
## select state, avg(pop) from (  
##     select state, city, sum(pop) as pop from zips group by state, city) zips 
## group by state
db.zips.aggregate( [
   { $group: { _id: { state: "$state", city: "$city" }, pop: { $sum: "$pop" } } },
   { $group: { _id: "$_id.state", avgCityPop: { $avg: "$pop" } } }
] )

## MongoDB UI
- https://www.mongodb.com/download-center?jmp=nav#compass

## 存取Facebook 資訊

In [2]:
ACCESSTOKEN = 'EAACEdEose0cBAEMFFplqcmwHZBqxZCpHfjMwt5Gs2maToSxfAwuP2FRM8WHIEbdeoT6h3sfDh7HFJFieMLnVPlzX2fhzQL38oMEzHXRZABuRTUSNPRhUkyUuVEMOrmhFAq0JXEOJK1oAJjYX8puyyyOYQM2ZBwM6XKhkHUSIKwZDZD'

In [6]:
import requests
res = requests.get('https://graph.facebook.com/v2.7/me/tagged_places?access_token={}'.format(ACCESSTOKEN))
print res.text

{"data":[{"id":"1664781083784462","created_time":"2016-03-09T06:42:31+0000","place":{"id":"182820988487172","location":{"city":"Singapore","country":"Singapore","latitude":25.040897308255,"longitude":121.55372061213,"street":"Singapore Changi Airport","zip":"81"},"name":"\u65b0\u52a0\u5761\u6a1f\u5b9c\u6a5f\u5834"}},{"id":"1664781047117799","created_time":"2016-03-09T06:42:08+0000","place":{"id":"171070619595488","location":{"city":"Ikeda-machi Kitaazumi-gun","country":"Japan","latitude":36.701882048976,"longitude":137.83747383417,"state":"Nagano","street":"\u767d\u99ac\u6751\u516b\u65b9","zip":"399-9301"},"name":"\u767d\u99ac \u516b\u65b9\u5c3e\u6839"}},{"id":"1664780843784486","created_time":"2016-03-09T06:40:18+0000","place":{"id":"150751781627629","location":{"city":"Taipei","country":"Taiwan","latitude":25.035434763842,"longitude":121.51971981538,"street":"No.21, Zhongshan S. Rd., Zhongzheng Dist., Taipei City 100, Taiwan (R.O.C.)","zip":"10048"},"name":"\u4e2d\u6b63\u7d00\u5ff5\u

In [14]:
import json
jd = json.loads(res.text) # 將json 轉變為 字典
place_ary = []
for rec in jd['data']:
    place_name = rec['place']['name']
    place_id = rec['place']['id']
    create_time = rec['created_time']
    place_ary.append({'place_id':place_id ,'place_name': place_name, 'created_time': create_time})

In [15]:
import pandas
df = pandas.DataFrame(place_ary)
df

,created_time,place_id,place_name
0,2016-03-09T06:42:31+0000,182820988487172,新加坡樟宜機場
1,2016-03-09T06:42:08+0000,171070619595488,白馬 八方尾根
2,2016-03-09T06:40:18+0000,150751781627629,中正紀念堂 Chiang Kai-shek Memorial Hall


## 抓取粉絲頁資訊

### 新增應用程式
- https://developers.facebook.com/apps/

In [17]:
ACCESSTOKEN = 'EAACEdEose0cBANzWCdFZAxntjyx2adJImM4lH2x3qektRTMYF5LOw6H1dedE6QCbqfJO16jaPgntb15KSb10F0MZCoysUKv1q6xFVtXCgEUHlkc098owQHIS0477EN23F7eXj8JKJexwQgdSwex7F3ZBveKPI6WA2LTZCCmFZBAZDZD'

In [25]:
fbid = '583984981774221'
fbsecret = '168718a9844764d5739090ecf036b28c'
ACCESSTOKEN = '{}|{}'.format(fbid, fbsecret)

In [26]:
import requests
res = requests.get('https://graph.facebook.com/v2.7/smartloan.tw/posts?access_token={}'.format(ACCESSTOKEN))
jd = json.loads(res.text) # 將json 轉變為 字典
place_ary = []
for rec in jd['data']:
    if 'message' in rec:
        message = rec['message'] 
    #place_name = rec['message']

## 建立與MongoDB 資料庫連線

In [31]:
import pymongo
try:
    conn=pymongo.MongoClient()
    print conn.database_names()
except pymongo.errors.ConnectionFailure as e:
    print "Could not connect to MongoDB: %s" % e

[u'local', u'test']


In [32]:
db= conn['fbdata']
print db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'fbdata')


In [35]:
print conn.database_names()

[u'local', u'test']


In [34]:
coll=db['posts']
print db.collection_names()

[]


In [37]:
import requests
res = requests.get('https://graph.facebook.com/v2.7/smartloan.tw/posts?access_token={}'.format(ACCESSTOKEN))
jd = json.loads(res.text)
coll.insert(jd['data'])

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


[ObjectId('57ef6576d18b5e1108f14909'),
 ObjectId('57ef6576d18b5e1108f1490a'),
 ObjectId('57ef6576d18b5e1108f1490b'),
 ObjectId('57ef6576d18b5e1108f1490c'),
 ObjectId('57ef6576d18b5e1108f1490d'),
 ObjectId('57ef6576d18b5e1108f1490e'),
 ObjectId('57ef6576d18b5e1108f1490f'),
 ObjectId('57ef6576d18b5e1108f14910'),
 ObjectId('57ef6576d18b5e1108f14911'),
 ObjectId('57ef6576d18b5e1108f14912'),
 ObjectId('57ef6576d18b5e1108f14913'),
 ObjectId('57ef6576d18b5e1108f14914'),
 ObjectId('57ef6576d18b5e1108f14915'),
 ObjectId('57ef6576d18b5e1108f14916'),
 ObjectId('57ef6576d18b5e1108f14917'),
 ObjectId('57ef6576d18b5e1108f14918'),
 ObjectId('57ef6576d18b5e1108f14919'),
 ObjectId('57ef6576d18b5e1108f1491a'),
 ObjectId('57ef6576d18b5e1108f1491b'),
 ObjectId('57ef6576d18b5e1108f1491c'),
 ObjectId('57ef6576d18b5e1108f1491d'),
 ObjectId('57ef6576d18b5e1108f1491e'),
 ObjectId('57ef6576d18b5e1108f1491f'),
 ObjectId('57ef6576d18b5e1108f14920'),
 ObjectId('57ef6576d18b5e1108f14921')]

In [38]:
print conn.database_names()

[u'fbdata', u'local', u'test']


In [39]:
print db.collection_names()

[u'posts']


## 塞打卡資料

In [42]:
import requests
import json
ACCESSTOKEN = 'EAACEdEose0cBAHdftyDXZBik81UTIuAT3zbS8ZAdxsZCaOIfOuhT4Fm15LFvhZButmQzcQ3PEIUiE5iLapPbdy9wGFbslsVQHN8LkkvGb3fZBsZAfXhdOI37d5bwEXYWkgHWdAGZBmhKtqmPZB9BTrx6Uto6eN9LBLVJFNrAsJjUwQZDZD'
res = requests.get('https://graph.facebook.com/v2.7/me/tagged_places?access_token={}'.format(ACCESSTOKEN))
jd = json.loads(res.text)
coll=db['tagged_places']
coll.insert(jd['data'])

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


[ObjectId('57ef66bbd18b5e1108f14922'),
 ObjectId('57ef66bbd18b5e1108f14923'),
 ObjectId('57ef66bbd18b5e1108f14924')]

In [45]:
coll = db['tagged_places']
coll.count()

3

In [46]:
coll = db['posts']
coll.count()

25